Validate implementation of *Expected Encompassing Intrinsic Bayes Factor* (EEIBF) normal mean hypothesis testing with unknown variance and an equality hypothesis.

The reference used to validate is the paper *Default Bayes Factors for Non-Nested Hypothesis Testing* by J. Berger and J. Mortera.

1: Berger, J. and J. Mortera (1999). Default bayes factors for nonnested hypothesis testing. Journal of the American Statistical Association 94 (446), 542–554.
Postscript for paper: https://www2.stat.duke.edu/~berger/papers/mortera.ps

## Notation
From [1] section 4, the three hypotheses are

$$
\begin{align*}
H_1&: \mu=0 \\
H_2&: \mu<0 \\
H_3&: \mu>0
\end{align*}
$$

and $H_0$ denotes the encompassing hypothesis (i.e. $\mu$ is unrestricted).

## Imports

In [1]:
import numpy as np
import scipy
from IPython.display import display, Math
from bbai.stat import NormalMeanHypothesis

## Sample Data

In [2]:
# define some sample data to validate the equations against
data = [
        -0.619487, -0.685728, -0.798718, 
        -0.353721, 0.36195, -0.0545616, 
        -1.09919, -1.28801, -1.30657
]
n = len(data)
mu_ml = np.mean(data)
sigma_ml = np.std(data)
print('mu_ml =', mu_ml)
print('sigma_ml =', sigma_ml)
hypothesis = NormalMeanHypothesis().test(data)

mu_ml = -0.6493372888888889
sigma_ml = 0.5302624236732443


## Uncorrected Factors

We'll first compute the uncorrected Bayes factors, $B_{i0}^N$. See equation (2) of [1].

The reference prior for the normal distribution with the mean, $\mu$, as the parameter of interest and unknown variance $\sigma^2$ is given by
$$
\pi(\mu, \sigma^2) = \frac{1}{\sigma^2}
$$

### $H_1$ uncorrected factor

$$
B_{10}^N = \frac{\int_0^\infty P(y\mid \mu=0, \sigma^2) \frac{1}{\sigma^2} d\sigma^2}{
\int_{-\infty}^\infty \int_0^\infty P(y\mid \mu, \sigma^2) \frac{1}{\sigma^2} d\sigma^2 d\mu
}
$$
Now,
$$
\int_0^\infty P(y \mid \mu, \sigma^2) \times \frac{1}{\sigma^2} \propto
   \mathcal{t}_{n-1}\left[ \sqrt{n-1} \left(\frac{\mu - \mu_{ml}}{\sigma_{ml}}\right)\right]
$$
where $\mathcal{t}_{n-1}$ is the PDF of the student t distribution with $n-1$ degrees of freedom.

Thus,
$$
B_{10}^N = \frac{\sqrt{n-1}}{\sigma_{ml}} \times \mathcal{t}_{n-1}\left[ \sqrt{n-1} \left(\frac{- \mu_{ml}}{\sigma_{ml}}\right)\right]
$$

In [3]:
def compute_B_10_N(y):
    n = len(y)
    t_dist = scipy.stats.t(n-1)
    mu_ml = np.mean(y)
    sigma_ml = np.std(y)
    return np.sqrt(n-1) / sigma_ml * t_dist.pdf(-np.sqrt(n - 1) * mu_ml / sigma_ml)

In [4]:
B_10_N = compute_B_10_N(data)
display(Math(r'B_{10}^N = ' + str(B_10_N) + r'\;\textrm{vs}\;' + str(hypothesis.equal_to_star_N)))

<IPython.core.display.Math object>

### $H_2$ uncorrected factor

$$
\begin{align*}
B_{20}^N &= \frac{\int_{-\infty}^0\int_0^\infty P(y\mid \mu, \sigma^2) \frac{1}{\sigma^2} d\sigma^2 d\mu}{
\int_{-\infty}^\infty \int_0^\infty P(y\mid \mu, \sigma^2) \frac{1}{\sigma^2} d\sigma^2 d\mu
} \\
         &= \frac{
                \int_{-\infty}^0 \mathcal{t}_{n-1}\left[\sqrt{n-1}\left(\frac{\mu - \mu_{ml}}{\sigma_{ml}}\right)\right]d\mu
             }{
                 \int_{-\infty}^\infty \mathcal{t}_{n-1}\left[\sqrt{n-1}\left(\frac{\mu - \mu_{ml}}{\sigma_{ml}}\right)\right]d\mu
             } \\
          &= \mathcal{T}_{n-1}\left[\sqrt{n-1}\left(\frac{- \mu_{ml}}{\sigma_{ml}}\right)\right]
\end{align*}
$$
where $\mathcal{T}_{n-1}$ represents the CDF of the Student t distribution with $n-1$ degrees of freedom

In [5]:
def compute_B_20_N(y):
    n = len(y)
    t_dist = scipy.stats.t(n-1)
    mu_ml = np.mean(y)
    sigma_ml = np.std(y)
    return t_dist.cdf(-np.sqrt(n - 1) * mu_ml / sigma_ml)

In [6]:
B_20_N = compute_B_20_N(data)
display(Math(r'B_{20}^N = ' + str(B_20_N) + r'\;\textrm{vs}\;' + str(hypothesis.left_to_star_N)))

<IPython.core.display.Math object>

### $H_3$ uncorrected factor

Similarly, for $H_3$, we have
$$
B_{30}^N = 1 - \mathcal{T}_{n-1}\left[\sqrt{n-1}\left(\frac{- \mu_{ml}}{\sigma_{ml}}\right)\right]
$$

In [7]:
def compute_B_30_N(y):
    return 1 - compute_B_20_N(y)

In [8]:
B_30_N = compute_B_30_N(data)
display(Math(r'B_{30}^N = ' + str(B_30_N) + r'\;\textrm{vs}\;' + str(hypothesis.right_to_star_N)))

<IPython.core.display.Math object>

## Correction Factors


From section 2.4.2 of [1], for testing the mean of normally distributed data $\boldsymbol{x}$ with unknown variance, the full EEIBF Bayes factor is given by

$$
B_{ji}^{EEI} = B_{ji}^N(\boldsymbol{x}) \times 
\frac{
 \mathbb{E}_\hat{\boldsymbol{\theta}}^{H_0} \left[B_{i0}^N(X_1, X_2)\right]
 }{
 \mathbb{E}_\hat{\boldsymbol{\theta}}^{H_0} \left[B_{j0}^N(X_1, X_2)\right]
 }
$$
where $X_1$ and $X_2$ denote independent random variables distributed under the
model for $H_0$ with parameters the $\hat{\boldsymbol{\theta}}=(\mu_{ml}, \sigma_{ml})$ that maximize the likelhood of the observed data $\boldsymbol{x}$.

bbai uses an efficient deterministic algorithm to compute the values
$$
\mathbb{E}_\hat{\boldsymbol{\theta}}^{H_0} \left[B_{i0}^N(X_1, X_2)\right]
$$
But to validate, we'll use the slower but more straightforward method of generating a bunch of random values.

In [9]:
# Generate some random values
np.random.seed(0)
N = 10000 
    # if we select N larger (e.g. 1000000), we'll get a closer match; 
    # but this should be reasonable for a ball park test
rx = np.random.normal(mu_ml, sigma_ml, size=(N, 2))

### $H_1$ correction factor

In [10]:
def compute_E_B_10_Xl(rx):
    N = len(rx)
    res = 0
    for xli in rx:
        res += compute_B_10_N(xli)
    return res / N

In [11]:
E_B_10_Xl = compute_E_B_10_Xl(rx)
display(Math(r'\mathbb{E}_{\boldsymbol{\theta}}^{H_0}\left[B_{10}^N(X_1, X_2)\right] = ' + str(E_B_10_Xl) + r'\;\textrm{vs}\;' + str(hypothesis.correction_equal)))

<IPython.core.display.Math object>

### $H_2$ correction factor

In [12]:
def compute_E_B_20_Xl(rx):
    N = len(rx)
    res = 0
    for xli in rx:
        res += compute_B_20_N(xli)
    return res / N

In [13]:
E_B_20_Xl = compute_E_B_20_Xl(rx)
display(Math(r'\mathbb{E}_{\boldsymbol{\theta}}^{H_0}\left[B_{20}^N(X_1, X_2)\right] = ' + str(E_B_20_Xl) + r'\;\textrm{vs}\;' + str(hypothesis.correction_left)))

<IPython.core.display.Math object>

### $H_3$ correction factor

In [14]:
def compute_E_B_30_Xl(rx):
    N = len(rx)
    res = 0
    for xli in rx:
        res += compute_B_30_N(xli)
    return res / N

In [15]:
E_B_30_Xl = compute_E_B_30_Xl(rx)
display(Math(r'\mathbb{E}_{\boldsymbol{\theta}}^{H_0}\left[B_{30}^N(X_1, X_2)\right] = ' + str(E_B_30_Xl) + r'\;\textrm{vs}\;' + str(hypothesis.correction_right)))

<IPython.core.display.Math object>

### Full Bayes Factors

$$
B_{i0}^{EEI} = B_{i0}^N(\boldsymbol{x}) \times 
\frac{
 1
 }{
 \mathbb{E}_\hat{\boldsymbol{\theta}}^{H_0} \left[B_{i0}^N(X_1, X_2)\right]
 }
$$

In [16]:
B_10 = B_10_N / E_B_10_Xl
display(Math(r'B_{10} = ' + str(B_10) + r'\;\textrm{vs}\;' + str(hypothesis.factor_equal)))

<IPython.core.display.Math object>

In [17]:
B_20 = B_20_N / E_B_20_Xl
display(Math(r'B_{20} = ' + str(B_20) + r'\;\textrm{vs}\;' + str(hypothesis.factor_left)))

<IPython.core.display.Math object>

In [18]:
B_30 = B_30_N / E_B_30_Xl
display(Math(r'B_{30} = ' + str(B_30) + r'\;\textrm{vs}\;' + str(hypothesis.factor_right)))

<IPython.core.display.Math object>

### Probabilities from factors

In [19]:
T = B_10 + B_20 + B_30
expected = np.array([B_10, B_20, B_30] / T)
actual = np.array([hypothesis.equal, hypothesis.left, hypothesis.right])
print(expected, 'vs', actual)

[0.09263094 0.88811406 0.019255  ] vs [0.08980186 0.89113729 0.01906085]
